In [3]:
import os
import requests
from datetime import datetime
import pandas as pd
import holidays
from amadeus import Client, ResponseError

## weather

In [2]:
def is_alert(code: int) -> bool:
    """
    Returns True if the weather condition code should trigger an alert, False otherwise.
    Optionally uses the description for more granularity.
    """

    # Thunderstorms (Group 2xx)
    if 200 <= code <= 232:
        return True  # All thunderstorms are potentially hazardous

    # Drizzle (Group 3xx)
    if 300 <= code <= 321:
        return False  # Generally light, not hazardous

    # Rain (Group 5xx)
    if code in {502, 503, 504, 511, 522, 531}:
        return True  # Heavy, extreme, or freezing rain
    elif 500 <= code <= 531:
        return False  # Light/moderate rain usually not alert-level

    # Snow (Group 6xx)
    if code in {602, 622, 616, 613, 620}:
        return True  # Heavy or mixed snow/ice conditions
    elif 600 <= code <= 622:
        return False  # Light snow/sleet usually advisory only

    # Atmosphere (Group 7xx)
    if code in {741, 762, 771, 781}:
        return True  # Fog, ash, squalls, tornado
    elif 700 <= code <= 781:
        return False  # Mist, haze, etc.

    # Clear & Clouds (Group 800-804)
    if code == 800:
        return False
    if 801 <= code <= 804:
        return False  # Cloud cover, no hazard

    return False  # Default to non-alert if unknown

In [3]:
city_coords = {
    "los angeles": (34.0522, -118.2437),
    "chicago": (41.8781, -87.6298),
    "houston": (29.7604, -95.3698),
    "denver": (39.7392, -104.9903),
    "newark": (40.7357, -74.1724),
    "san francisco": (37.7749, -122.4194),
    "washington dc": (38.9072, -77.0369),
 
}

In [4]:
coords = [value for key, value in city_coords.items()]
coords

[(34.0522, -118.2437),
 (41.8781, -87.6298),
 (29.7604, -95.3698),
 (39.7392, -104.9903),
 (40.7357, -74.1724),
 (37.7749, -122.4194),
 (38.9072, -77.0369)]

## city

In [5]:
df_city = pd.DataFrame({
    'id': range(1, len(city_coords)+1),
    'city_name': city_coords.keys()})

df_city

,id,city_name
0,1,los angeles
1,2,chicago
2,3,houston
3,4,denver
4,5,newark
5,6,san francisco
6,7,washington dc


In [6]:
df_city.to_csv('data/intermediate/city.csv', index=False)

### generate datetime

In [7]:


def get_city_id(df):
    def find_city(lat, lon, lookup):
        target = (lat, lon)
        return next((k for k, v in lookup.items() if v == target), None)
    
    city_to_id = dict(zip(df_city['city_name'], df_city['id']))
    df['city'] = df.apply(lambda row: find_city(row["lat"], row["lon"], city_coords), axis=1)
    df['city_id'] = df['city'].map(city_to_id)
    df = df.drop(columns='city')

    return df

In [8]:
from datetime import datetime, timedelta, timezone

# Start and end dates
start_date = datetime(2025, 1, 1, tzinfo=timezone.utc)
end_date = datetime(2025, 6, 22, tzinfo=timezone.utc)

# Generate list of timestamps at 00:00:00 UTC each day
timestamps = []
current_date = start_date
while current_date <= end_date:
    timestamps.append(int(current_date.timestamp()))
    current_date += timedelta(days=1)

print(timestamps)

[1735689600, 1735776000, 1735862400, 1735948800, 1736035200, 1736121600, 1736208000, 1736294400, 1736380800, 1736467200, 1736553600, 1736640000, 1736726400, 1736812800, 1736899200, 1736985600, 1737072000, 1737158400, 1737244800, 1737331200, 1737417600, 1737504000, 1737590400, 1737676800, 1737763200, 1737849600, 1737936000, 1738022400, 1738108800, 1738195200, 1738281600, 1738368000, 1738454400, 1738540800, 1738627200, 1738713600, 1738800000, 1738886400, 1738972800, 1739059200, 1739145600, 1739232000, 1739318400, 1739404800, 1739491200, 1739577600, 1739664000, 1739750400, 1739836800, 1739923200, 1740009600, 1740096000, 1740182400, 1740268800, 1740355200, 1740441600, 1740528000, 1740614400, 1740700800, 1740787200, 1740873600, 1740960000, 1741046400, 1741132800, 1741219200, 1741305600, 1741392000, 1741478400, 1741564800, 1741651200, 1741737600, 1741824000, 1741910400, 1741996800, 1742083200, 1742169600, 1742256000, 1742342400, 1742428800, 1742515200, 1742601600, 1742688000, 1742774400, 174

In [9]:
# Create DataFrames for city*date
df_coords = pd.DataFrame(coords, columns=["lat", "lon"])
df_times = pd.DataFrame(timestamps, columns=["timestamp"])

# Add keys to enable cartesian join
df_coords["key"] = 1
df_times["key"] = 1

# Cartesian product via merge
df_coord_time = pd.merge(df_coords, df_times, on="key").drop("key", axis=1)

# Optional: convert timestamp to readable date
df_coord_time["datetime"] = pd.to_datetime(df_coord_time["timestamp"], unit="s")

print(df_coord_time)

          lat       lon   timestamp   datetime
0     34.0522 -118.2437  1735689600 2025-01-01
1     34.0522 -118.2437  1735776000 2025-01-02
2     34.0522 -118.2437  1735862400 2025-01-03
3     34.0522 -118.2437  1735948800 2025-01-04
4     34.0522 -118.2437  1736035200 2025-01-05
...       ...       ...         ...        ...
1206  38.9072  -77.0369  1750204800 2025-06-18
1207  38.9072  -77.0369  1750291200 2025-06-19
1208  38.9072  -77.0369  1750377600 2025-06-20
1209  38.9072  -77.0369  1750464000 2025-06-21
1210  38.9072  -77.0369  1750550400 2025-06-22

[1211 rows x 4 columns]


In [10]:
weather_api = os.getenv('OpenWeatherMap_API')

In [11]:
# weather_api = os.getenv('OpenWeatherMap_API')


# url = f"https://api.openweathermap.org/data/3.0/onecall/timemachine?lat=39.099724&lon=-94.578331&dt=1750550400&&appid={weather_api}"

# response = requests.get(url)

# if response.status_code == 200:
#     data_w = response.json()
#     # print("Current temperature:", data['current']['temp'])
#     print(data_w)
# else:
#     print("Error:", response.status_code, response.text)

In [ ]:
# import time
# import requests
# import pandas as pd

MAX_CALLS_PER_MINUTE = 60
MAX_TOTAL_CALLS = 500
SECONDS_BETWEEN_CALLS = 60 / MAX_CALLS_PER_MINUTE  # 1.0 seconds

total_calls = 0
start_time = time.time()
df_w = pd.DataFrame()  # Initialize your result dataframe

for index, row in df_coord_time.iterrows():
    if total_calls >= MAX_TOTAL_CALLS:
        print("Reached total API call limit (500).")
        break

    lat = row['lat']
    lon = row['lon']
    timestamp = row['timestamp']
    url = f"https://api.openweathermap.org/data/3.0/onecall/timemachine?lat={lat}&lon={lon}&dt={timestamp}&appid={weather_api}"

    response = requests.get(url)
    total_calls += 1

    if response.status_code == 200:
        data_w = response.json()
        print(index, data_w)
        
        weather_list = data_w['data'][0]['weather']
        records = [{
            'lat': data_w['lat'],
            'lon': data_w['lon'],
            'date': data_w['data'][0]['dt'],
            'weather_id': w['id'],
            'weather': w['main']
        } for w in weather_list]

        df_w = pd.concat([df_w, pd.DataFrame(records)], ignore_index=True)
    else:
        print("Error:", response.status_code, response.text)

    # Rate limit: wait to avoid >60 calls/min
    time.sleep(SECONDS_BETWEEN_CALLS)

In [ ]:


# # Apply it row-wise

# # df_w['utc_date'] = pd.to_datetime(df_w['date'], unit='s', utc=True)
# # df_w['date'] = pd.to_datetime(df_w['date'], unit='s').dt.date
# df_weather = df_w.copy()
# df_weather['date'] = pd.to_datetime(df_w['date'], unit='s').dt.strftime('%Y-%m-%d')
# df_weather['alert'] = df_weather['weather_id'].apply(lambda x: is_alert(x))
# df_weather = get_city_id(df_weather)

# df_weather['id'] = range(1, len(df_weather)+1)

# df_weather = df_weather.drop(['lat', 'lon'], axis=1)

# # adjusted order
# desired_order = ['id', 'date', 'city_id', 'weather_id', 'weather', 'alert']
# df_weather = df_weather[desired_order]

# df_weather

In [15]:
# write out
# df_weather.to_csv('data/intermediate/weather_500.csv', index=False)

In [16]:
filtered_df = df_coord_time[515:]
filtered_df 

,lat,lon,timestamp,datetime
515,29.7604,-95.3698,1750291200,2025-06-19
516,29.7604,-95.3698,1750377600,2025-06-20
517,29.7604,-95.3698,1750464000,2025-06-21
518,29.7604,-95.3698,1750550400,2025-06-22
519,39.7392,-104.9903,1735689600,2025-01-01
...,...,...,...,...
1206,38.9072,-77.0369,1750204800,2025-06-18
1207,38.9072,-77.0369,1750291200,2025-06-19
1208,38.9072,-77.0369,1750377600,2025-06-20
1209,38.9072,-77.0369,1750464000,2025-06-21


# weather api 500:

In [39]:
# continue the scraping
weather_api = os.getenv('OpenWeatherMap_API')

import time
import requests
import pandas as pd
from requests.exceptions import RequestException, ConnectionError


MAX_CALLS_PER_MINUTE = 60
MAX_TOTAL_CALLS = 1000
SECONDS_BETWEEN_CALLS = 60 / MAX_CALLS_PER_MINUTE  # 1.0 seconds

total_calls = 0
start_time = time.time()
df_w = pd.DataFrame()  # Initialize your result dataframe

try:
    for index, row in df_coord_time[501:514].iterrows():
        if total_calls >= MAX_TOTAL_CALLS:
            print("Reached total API call limit (1000).")
            break

        lat = row['lat']
        lon = row['lon']
        timestamp = row['timestamp']
        url = f"https://api.openweathermap.org/data/3.0/onecall/timemachine?lat={lat}&lon={lon}&dt={timestamp}&appid={weather_api}"

        try:
            response = requests.get(url, timeout=10)
            total_calls += 1

            if response.status_code == 200:
                data_w = response.json()
                print(index, data_w)

                weather_list = data_w['data'][0]['weather']
                records = [{
                    'lat': data_w['lat'],
                    'lon': data_w['lon'],
                    'date': data_w['data'][0]['dt'],
                    'weather_id': w['id'],
                    'weather': w['main']
                } for w in weather_list]

                df_w = pd.concat([df_w, pd.DataFrame(records)], ignore_index=True)

            else:
                print("Error:", response.status_code, response.text)

            time.sleep(SECONDS_BETWEEN_CALLS)

        except (ConnectionError, RequestException) as e:
            print(f"⚠️ Request failed at index {index}: {e}")
            break

except Exception as e:
    print(f"🚨 Fatal error during loop: {e}")

501 {'lat': 29.7604, 'lon': -95.3698, 'timezone': 'America/Chicago', 'timezone_offset': -18000, 'data': [{'dt': 1749081600, 'sunrise': 1749036060, 'sunset': 1749086343, 'temp': 300.57, 'feels_like': 304.98, 'pressure': 1012, 'humidity': 89, 'dew_point': 298.59, 'clouds': 63, 'wind_speed': 2.24, 'wind_deg': 106, 'wind_gust': 4.92, 'weather': [{'id': 501, 'main': 'Rain', 'description': 'moderate rain', 'icon': '10d'}], 'rain': {'1h': 3.56}}]}
502 {'lat': 29.7604, 'lon': -95.3698, 'timezone': 'America/Chicago', 'timezone_offset': -18000, 'data': [{'dt': 1749168000, 'sunrise': 1749122451, 'sunset': 1749172772, 'temp': 304.66, 'feels_like': 311.66, 'pressure': 1013, 'humidity': 71, 'dew_point': 298.76, 'clouds': 40, 'visibility': 10000, 'wind_speed': 5.66, 'wind_deg': 180, 'weather': [{'id': 802, 'main': 'Clouds', 'description': 'scattered clouds', 'icon': '03d'}]}]}
503 {'lat': 29.7604, 'lon': -95.3698, 'timezone': 'America/Chicago', 'timezone_offset': -18000, 'data': [{'dt': 1749254400, '

In [22]:
# continue the scraping
weather_api = os.getenv('OpenWeatherMap_API')

import time
import requests
import pandas as pd
from requests.exceptions import RequestException, ConnectionError


MAX_CALLS_PER_MINUTE = 60
MAX_TOTAL_CALLS = 1000
SECONDS_BETWEEN_CALLS = 60 / MAX_CALLS_PER_MINUTE  # 1.0 seconds

total_calls = 0
start_time = time.time()
df_w = pd.DataFrame()  # Initialize your result dataframe

try:
    for index, row in df_coord_time[515:].iterrows():
        if total_calls >= MAX_TOTAL_CALLS:
            print("Reached total API call limit (1000).")
            break

        lat = row['lat']
        lon = row['lon']
        timestamp = row['timestamp']
        url = f"https://api.openweathermap.org/data/3.0/onecall/timemachine?lat={lat}&lon={lon}&dt={timestamp}&appid={weather_api}"

        try:
            response = requests.get(url, timeout=10)
            total_calls += 1

            if response.status_code == 200:
                data_w = response.json()
                print(index, data_w)

                weather_list = data_w['data'][0]['weather']
                records = [{
                    'lat': data_w['lat'],
                    'lon': data_w['lon'],
                    'date': data_w['data'][0]['dt'],
                    'weather_id': w['id'],
                    'weather': w['main']
                } for w in weather_list]

                df_w = pd.concat([df_w, pd.DataFrame(records)], ignore_index=True)

            else:
                print("Error:", response.status_code, response.text)

            time.sleep(SECONDS_BETWEEN_CALLS)

        except (ConnectionError, RequestException) as e:
            print(f"⚠️ Request failed at index {index}: {e}")
            break

except Exception as e:
    print(f"🚨 Fatal error during loop: {e}")

515 {'lat': 29.7604, 'lon': -95.3698, 'timezone': 'America/Chicago', 'timezone_offset': -18000, 'data': [{'dt': 1750291200, 'sunrise': 1750245662, 'sunset': 1750296261, 'temp': 304.83, 'feels_like': 311.63, 'pressure': 1012, 'humidity': 67, 'dew_point': 297.95, 'uvi': 0.54, 'clouds': 0, 'visibility': 10000, 'wind_speed': 10.29, 'wind_deg': 180, 'wind_gust': 13.38, 'weather': [{'id': 800, 'main': 'Clear', 'description': 'clear sky', 'icon': '01d'}]}]}
516 {'lat': 29.7604, 'lon': -95.3698, 'timezone': 'America/Chicago', 'timezone_offset': -18000, 'data': [{'dt': 1750377600, 'sunrise': 1750332072, 'sunset': 1750382676, 'temp': 304.89, 'feels_like': 311.89, 'pressure': 1014, 'humidity': 72, 'dew_point': 299.21, 'uvi': 0.51, 'clouds': 0, 'visibility': 10000, 'wind_speed': 7.2, 'wind_deg': 150, 'wind_gust': 10.29, 'weather': [{'id': 800, 'main': 'Clear', 'description': 'clear sky', 'icon': '01d'}]}]}
517 {'lat': 29.7604, 'lon': -95.3698, 'timezone': 'America/Chicago', 'timezone_offset': -180

In [40]:
# Apply it row-wise

# df_w['utc_date'] = pd.to_datetime(df_w['date'], unit='s', utc=True)
# df_w['date'] = pd.to_datetime(df_w['date'], unit='s').dt.date
df_weather = df_w.copy()
df_weather['date'] = pd.to_datetime(df_w['date'], unit='s').dt.strftime('%Y-%m-%d')
df_weather['alert'] = df_weather['weather_id'].apply(lambda x: is_alert(x))
df_weather = get_city_id(df_weather)

df_weather['id'] = range(1, len(df_weather)+1)

df_weather = df_weather.drop(['lat', 'lon'], axis=1)

# adjusted order
desired_order = ['id', 'date', 'city_id', 'weather_id', 'weather', 'alert']
df_weather = df_weather[desired_order]

df_weather

,id,date,city_id,weather_id,weather,alert
0,1,2025-06-05,3,501,Rain,False
1,2,2025-06-06,3,802,Clouds,False
2,3,2025-06-07,3,800,Clear,False
3,4,2025-06-08,3,801,Clouds,False
4,5,2025-06-09,3,802,Clouds,False
5,6,2025-06-10,3,500,Rain,False
6,7,2025-06-11,3,803,Clouds,False
7,8,2025-06-12,3,801,Clouds,False
8,9,2025-06-13,3,800,Clear,False
9,10,2025-06-14,3,800,Clear,False


In [ ]:
weather_500 = pd.read_csv('data/intermediate/weather.csv')
weather_500 = weather_500.iloc[:, 1:]

In [45]:
df_weather_all = pd.concat([weather_500, df_weather])
df_weather_all

,id,date,city_id,weather_id,weather,alert
0,1,2025-01-01,1,804,Clouds,False
1,2,2025-01-02,1,800,Clear,False
2,3,2025-01-03,1,800,Clear,False
3,4,2025-01-04,1,800,Clear,False
4,5,2025-01-05,1,804,Clouds,False
...,...,...,...,...,...,...
8,9,2025-06-13,3,800,Clear,False
9,10,2025-06-14,3,800,Clear,False
10,11,2025-06-15,3,803,Clouds,False
11,12,2025-06-16,3,804,Clouds,False


In [46]:
df_weather_all.to_csv('data/intermediate/weather.csv', index=False)

In [ ]:
# from geopy.geocoders import Nominatim

# def get_city_name(lat, lon):
#     geolocator = Nominatim(user_agent="weather_alert_app")
#     location = geolocator.reverse((lat, lon), language="en")
#     if location and 'address' in location.raw:
#         address = location.raw['address']
#         # return address.get('city') or address.get('town') or address.get('village') or address.get('state')
#         return address.get('city')
#     return None

# # Example usage:
# city = get_city_name(39.099724, -94.578331)  # Washington, DC
# print(city)

In [ ]:
# Extracting relevant data

# weather_list = data_w['data'][0]['weather']

# # Build a list of records for each weather condition
# records = [{'lat': data_w['lat'], 'lon': data_w['lon'], 'date': data_w['data'][0]['dt'], 'weather_id': w['id'], 'weather': w['main']} for w in weather_list]

# # Convert to DataFrame
# df_w = pd.DataFrame(records)
# print(df_w)

### national holidays

In [ ]:
# cal_api = os.getenv('Calendarific_API')
# url = f"https://calendarific.com/api/v2/holidays?api_key={cal_api}&country=US&year=2025"

# response = requests.get(url)

# if response.status_code == 200:
#     data_h = response.json()
#     # print("Current temperature:", data['current']['temp'])
#     print(data_h)
# else:
#     print("Error:", response.status_code, response.text)

In [ ]:
# df_holiday = pd.DataFrame(data_h['response']['holidays'])
# df_holiday

In [ ]:
# for ptype in df_holiday['primary_type'].unique():
#     print(f"\n--- {ptype} ---")
#     print(df_holiday[df_holiday['primary_type'] == ptype]['name'].unique())


In [ ]:
# df_holiday = df_holiday[df_holiday['primary_type'].isin(['Federal Holiday', 'State Holiday'])][['name', 'locations']]
# df_holiday = df_holiday[df_holiday['primary_type'].isin(['Federal Holiday', 'State Holiday'])][['name', 'locations']]

In [ ]:
# # Example: merging 'locations' column by 'name' using newline separator
# merged = df_holiday.groupby('name', as_index=False).agg({
#     'locations': lambda x: '\\'.join(str(v) for v in x)
#     # For non-text columns you might want to keep the first or max, etc.
#     # 'date': 'first',  # Or 'min', 'max', etc., depending on your goal
# })

# merged

In [ ]:
# def keep_all_location(group):
#     if 'All' in group['locations'].values:
#         return group[group['locations'] == 'All']
#     else:
#         return group

# result = df_holiday.groupby('name', group_keys=False).apply(keep_all_location)
# result

In [ ]:
# # data placeholder
# df_holidays = pd.DataFrame()

# # get 23-25 holiday data
# for year in [2023, 2024, 2025]:
#     us_holidays = holidays.UnitedStates(years=year)
#     df_holidays = pd.concat([df_holidays, pd.DataFrame(list(us_holidays.items()), columns=['date', 'name'])], ignore_index=True)

# # add id column
# df_holidays['id'] = range(1, len(df_holidays)+1)
# df_holidays['date'] = pd.to_datetime(df_holidays['date'], errors='coerce')

# # adjust order
# desired_order = ['id', 'name', 'date']
# df_holidays = df_holidays[desired_order]
# df_holidays['state'] = 'placeholder'

# df_holidays

## airport

In [ ]:
# price_api = os.getenv('AMADEUS_API_KEY')
# price_secret = os.getenv('AMADEUS_SECRET')

# amadeus = Client(
#     client_id=price_api,
#     client_secret=price_secret
# )



# # city - airport
# iata_results = []


# for city, (lat, lon) in city_coords.items():
#     try:
#         response = amadeus.reference_data.locations.airports.get(latitude=lat, longitude=lon)
#         if response.data:
#             nearest = response.data[0]
#             iata_results.append({
#                 "iata_code": nearest["iataCode"]
#             })
#     except Exception as e:
#         print(f"Failed for {city}: {e}")

# # Convert to DataFrame
# df_iata = pd.DataFrame(iata_results)
# df_iata['city_id'] = range(1, len(df_iata)+1)
# df_iata['id'] = range(1, len(df_iata)+1)
# desired_order = ['id', 'iata_code', 'city_id']
# df_iata = df_iata[desired_order]

# print(df_iata)

   id iata_code  city_id
0   1       LAX        1
1   2       ORD        2
2   3       IAH        3
3   4       DEN        4
4   5       EWR        5
5   6       SFO        6
6   7       DCA        7


In [ ]:
# encode dc airport to united hub
# df_iata.loc[6, 'iata_code'] = 'IAD'
# df_iata.to_csv('data/intermediate/city_airport.csv', index=False)
df_iata = pd.read_csv('data/intermediate/city_airport.csv')
df_iata

TypeError: read_csv() got an unexpected keyword argument 'index'

## route

In [24]:
from itertools import permutations

# Generate all ordered pairs (departure ≠ destination)
routes = list(permutations(df_iata["id"], 2))

# Convert to DataFrame
routes_df = pd.DataFrame(routes, columns=["depar_airport_id", "desti_airport_id"])

# Optional: Add a unique route_id (if you'll insert into SQL table)
routes_df["id"] = range(1, len(routes_df) + 1)
desired_order = ['id', "depar_airport_id", "desti_airport_id"]
routes_df = routes_df[desired_order]

routes_df

,id,depar_airport_id,desti_airport_id
0,1,1,2
1,2,1,3
2,3,1,4
3,4,1,5
4,5,1,6
5,6,1,7
6,7,2,1
7,8,2,3
8,9,2,4
9,10,2,5


In [150]:
routes_df.to_csv('data/intermediate/routes.csv', index=False)

## flight

In [ ]:
# !pip install amadeus

In [92]:
def airport_to_route(df):
    # Create mapping from IATA to airport ID
    iata_to_id = df_iata.set_index('iata_code')['id'].to_dict()

    # Map airport IDs to flights
    df['depar_airport_id'] = df['origin'].map(iata_to_id)
    df['arrival_airport_id'] = df['destination'].map(iata_to_id)

    # Create route lookup: {(depart_id, arrive_id): route_id}
    route_lookup = {
        (row['depar_airport_id'], row['desti_airport_id']): row['id']
        for _, row in routes_df.iterrows()
    }

    # Apply to get route_id
    df['route_id'] = df.apply(
        lambda row: route_lookup.get((row['depar_airport_id'], row['arrival_airport_id'])),
        axis=1
    )
    print(df)
    return(df)

In [ ]:
# historical price trial
# from amadeus import Client, ResponseError

# price_api = os.getenv('AMADEUS_API_KEY')
# price_secret = os.getenv('AMADEUS_SECRET')

# amadeus = Client(
#     client_id=price_api,
#     client_secret=price_secret
# )

# try: 
#     response = amadeus.analytics.itinerary_price_metrics.get(originIataCode='ORD', destinationIataCode='EWR',
#                                             departureDate='2025-05-01')
# except ResponseError as error:
#     print(error)

# response.data

[]

In [87]:
airport_route = pd.DataFrame(list(permutations(df_iata["iata_code"], 2)))
# airport_route

dates = pd.date_range(start="2025-01-01", end="2025-06-30", freq="M", tz="UTC")
df_dates = pd.DataFrame(dates)

route_dates = airport_route.merge(df_dates, how="cross")
route_dates.columns = ['departure_iata', 'arrival_iata', 'departure_date']
route_dates['departure_date'] = pd.to_datetime(route_dates['departure_date']).dt.strftime('%Y-%m-%d')
route_dates['id'] = range(1, len(route_dates)+1)

route_dates

/var/folders/x7/k10zkcs917x1mtwm_l5s0z5c0000gn/T/ipykernel_66000/736925844.py:4: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  dates = pd.date_range(start="2025-01-01", end="2025-06-30", freq="M", tz="UTC")


,departure_iata,arrival_iata,departure_date,id
0,LAX,ORD,2025-01-31,1
1,LAX,ORD,2025-02-28,2
2,LAX,ORD,2025-03-31,3
3,LAX,ORD,2025-04-30,4
4,LAX,ORD,2025-05-31,5
...,...,...,...,...
247,IAD,SFO,2025-02-28,248
248,IAD,SFO,2025-03-31,249
249,IAD,SFO,2025-04-30,250
250,IAD,SFO,2025-05-31,251


In [ ]:
from amadeus import Client, ResponseError

price_api = os.getenv('AMADEUS_API_KEY')
price_secret = os.getenv('AMADEUS_SECRET')

amadeus = Client(
    client_id=price_api,
    client_secret=price_secret
)
# Quartile label mapping
quartile_map = {
    "MINIMUM": "price_quantile_minimum",
    "FIRST": "price_quantile_low",
    "MEDIUM": "price_quantile_middle",
    "THIRD": "price_quantile_high",
    "MAXIMUM": "price_quantile_maximum"
}

MAX_CALLS_PER_MIN = 20
SECONDS_BETWEEN_CALLS = 60 / MAX_CALLS_PER_MIN  # 3 seconds
MAX_TOTAL_CALLS = 2000

response_data = []
call_count = 0

for index, route in route_dates.iterrows():
    if call_count >= MAX_TOTAL_CALLS:
        print("Reached monthly API call limit (2000). Stopping.")
        break

    departure_iata = route['departure_iata']
    arrival_iata = route['arrival_iata']
    departure_date = route['departure_date']

    try: 
        response = amadeus.analytics.itinerary_price_metrics.get(
            originIataCode=departure_iata, 
            destinationIataCode=arrival_iata,
            departureDate=departure_date,
            currencyCode="USD"
        )

        call_count += 1

        if response.data:
            response_data.append(response.data[0])
            print(index, response.data[0])
        else:
            print(f'No data for {departure_iata} to {arrival_iata} on {departure_date}')

        # Respect per-minute rate limit
        time.sleep(SECONDS_BETWEEN_CALLS)

    except ResponseError as error:
        if hasattr(error, 'response') and error.response.status_code == 429:
            retry_after = int(error.response.headers.get("Retry-After", 60))
            print(f"429 Rate Limit hit. Retrying after {retry_after} seconds.")
            time.sleep(retry_after)
        else:
            print(f"Error for {departure_iata} to {arrival_iata} on {departure_date}. ")

# Build DataFrame from response_data
rows = []
for item in response_data:
    row = {
        "origin": item['origin']['iataCode'],
        "destination": item['destination']['iataCode'],
        "departure_date": item['departureDate']
    }
    for metric in item['priceMetrics']:
        key = quartile_map.get(metric['quartileRanking'])
        row[key] = float(metric['amount'])
        
    rows.append(row)

df_f = pd.DataFrame(rows)
print(df_f)

0 {'type': 'itinerary-price-metric', 'origin': {'iataCode': 'LAX'}, 'destination': {'iataCode': 'ORD'}, 'departureDate': '2025-01-31', 'transportType': 'FLIGHT', 'currencyCode': 'USD', 'oneWay': False, 'priceMetrics': [{'amount': '45.06', 'quartileRanking': 'MINIMUM'}, {'amount': '225.63', 'quartileRanking': 'FIRST'}, {'amount': '323.14', 'quartileRanking': 'MEDIUM'}, {'amount': '417.43', 'quartileRanking': 'THIRD'}, {'amount': '590.70', 'quartileRanking': 'MAXIMUM'}]}
1 {'type': 'itinerary-price-metric', 'origin': {'iataCode': 'LAX'}, 'destination': {'iataCode': 'ORD'}, 'departureDate': '2025-02-28', 'transportType': 'FLIGHT', 'currencyCode': 'USD', 'oneWay': False, 'priceMetrics': [{'amount': '45.06', 'quartileRanking': 'MINIMUM'}, {'amount': '231.31', 'quartileRanking': 'FIRST'}, {'amount': '327.49', 'quartileRanking': 'MEDIUM'}, {'amount': '428.40', 'quartileRanking': 'THIRD'}, {'amount': '588.68', 'quartileRanking': 'MAXIMUM'}]}
2 {'type': 'itinerary-price-metric', 'origin': {'iat

In [129]:
df_flight = airport_to_route(df_f)
df_flight = df_flight.iloc[:, 2:]
df_flight['departure_date'] = pd.to_datetime(df_flight['departure_date'])
df_flight['id'] = range(1, len(df_flight)+1)
desired_order = ['id', 'route_id', 'departure_date', 'price_quantile_minimum', 'price_quantile_low', 'price_quantile_middle', 'price_quantile_high', 'price_quantile_maximum']
df_flight = df_flight[desired_order]
# print(df_flight)
df_flight

    origin destination departure_date  price_quantile_minimum  \
0      LAX         ORD     2025-01-31                   45.06   
1      LAX         ORD     2025-02-28                   45.06   
2      LAX         ORD     2025-03-31                   45.06   
3      LAX         ORD     2025-04-30                   45.50   
4      LAX         ORD     2025-05-31                   45.48   
..     ...         ...            ...                     ...   
121    IAD         LAX     2025-02-28                  100.54   
122    IAD         LAX     2025-03-31                  100.54   
123    IAD         LAX     2025-04-30                  104.10   
124    IAD         LAX     2025-05-31                  104.65   
125    IAD         LAX     2025-06-30                  105.97   

     price_quantile_low  price_quantile_middle  price_quantile_high  \
0                225.63                 323.14               417.43   
1                231.31                 327.49               428.40   
2     

,id,route_id,departure_date,price_quantile_minimum,price_quantile_low,price_quantile_middle,price_quantile_high,price_quantile_maximum
0,1,1,2025-01-31,45.06,225.63,323.14,417.43,590.70
1,2,1,2025-02-28,45.06,231.31,327.49,428.40,588.68
2,3,1,2025-03-31,45.06,241.29,342.78,434.55,595.02
3,4,1,2025-04-30,45.50,268.53,364.22,491.29,602.88
4,5,1,2025-05-31,45.48,269.15,366.57,494.52,603.35
...,...,...,...,...,...,...,...,...
121,122,37,2025-02-28,100.54,430.24,454.27,630.83,653.03
122,123,37,2025-03-31,100.54,433.24,483.73,659.16,664.23
123,124,37,2025-04-30,104.10,428.52,483.82,658.88,668.86
124,125,37,2025-05-31,104.65,424.84,491.99,655.11,657.23


In [ ]:
# # Expand rows
# expanded_rows = []

# # Iterate over each row and expand by days in the month
# for _, row in df_flight.iterrows():
#     last_day = row['departure_date']
#     first_day = last_day.replace(day=1)
    
#     # Generate all days in that month
#     for day in pd.date_range(start=first_day, end=last_day):
#         new_row = row.copy()
#         new_row['departure_date'] = first_day  # replace with first day of month
#         new_row['date'] = day                   # new column with each day of month
#         expanded_rows.append(new_row)


# # Overwrite df_flight with expanded version
# df_flight = pd.DataFrame(expanded_rows)
# df_flight = df_flight.drop(['date'], axis=1)
# df_flight['id'] = range(1, len(df_flight)+1)
# desired_order = ['id', 'route_id', 'departure_date', 'price_quantile_minimum', 'price_quantile_low', 'price_quantile_middle', 'price_quantile_high', 'price_quantile_maximum']
# df_flight = df_flight[desired_order]
# print(df_flight)

       id  route_id departure_date  price_quantile_minimum  \
0       1         1     2025-01-01                   45.06   
0       2         1     2025-01-01                   45.06   
0       3         1     2025-01-01                   45.06   
0       4         1     2025-01-01                   45.06   
0       5         1     2025-01-01                   45.06   
..    ...       ...            ...                     ...   
125  3797        37     2025-06-01                  105.97   
125  3798        37     2025-06-01                  105.97   
125  3799        37     2025-06-01                  105.97   
125  3800        37     2025-06-01                  105.97   
125  3801        37     2025-06-01                  105.97   

     price_quantile_low  price_quantile_middle  price_quantile_high  \
0                225.63                 323.14               417.43   
0                225.63                 323.14               417.43   
0                225.63                 32

In [130]:
df_flight.to_csv('data/intermediate/flight_month.csv', index=False)

In [131]:
df_flight = pd.read_csv('data/intermediate/flight_month.csv')
df_flight

,id,route_id,departure_date,price_quantile_minimum,price_quantile_low,price_quantile_middle,price_quantile_high,price_quantile_maximum
0,1,1,2025-01-31,45.06,225.63,323.14,417.43,590.70
1,2,1,2025-02-28,45.06,231.31,327.49,428.40,588.68
2,3,1,2025-03-31,45.06,241.29,342.78,434.55,595.02
3,4,1,2025-04-30,45.50,268.53,364.22,491.29,602.88
4,5,1,2025-05-31,45.48,269.15,366.57,494.52,603.35
...,...,...,...,...,...,...,...,...
121,122,37,2025-02-28,100.54,430.24,454.27,630.83,653.03
122,123,37,2025-03-31,100.54,433.24,483.73,659.16,664.23
123,124,37,2025-04-30,104.10,428.52,483.82,658.88,668.86
124,125,37,2025-05-31,104.65,424.84,491.99,655.11,657.23


In [121]:
from pandas.tseries.offsets import MonthEnd

def expand_to_full_month(row):
    start_date = row['departure_date']
    end_date = start_date + MonthEnd(0)  # get last day of the month
    
    # Generate all dates from start_date to end_date inclusive
    all_dates = pd.date_range(start=start_date, end=end_date)
    
    # Repeat the row for each day and assign the dates
    expanded_rows = pd.DataFrame({
        **{col: [row[col]] * len(all_dates) for col in row.index if col != 'departure_date'},
        'departure_date': all_dates
    })
    
    return expanded_rows

# Apply to all rows and concatenate
expanded_df = pd.concat(df_flight.apply(expand_to_full_month, axis=1).to_list(), ignore_index=True)

print(expanded_df)

     price_quantile_minimum  price_quantile_low  price_quantile_middle  \
0                     45.06              225.63                 323.14   
1                     45.06              231.31                 327.49   
2                     45.06              241.29                 342.78   
3                     45.50              268.53                 364.22   
4                     45.48              269.15                 366.57   
..                      ...                 ...                    ...   
121                  100.54              430.24                 454.27   
122                  100.54              433.24                 483.73   
123                  104.10              428.52                 483.82   
124                  104.65              424.84                 491.99   
125                  105.97              352.44                 467.92   

     price_quantile_high  price_quantile_maximum  depar_airport_id  \
0                 417.43                 

In [ ]:
# import pandas as pd


# def parse_flight_price(flight_offers: list) -> pd.DataFrame:
#     rows = []

#     for offer in flight_offers:
#         total_price = offer.get("price", {}).get("grandTotal")
#         traveler_pricing = offer.get("travelerPricings", [{}])[0]
#         cabin = traveler_pricing.get("fareDetailsBySegment", [{}])[0].get("cabin")

#         for itinerary in offer.get("itineraries", []):
#             for segment in itinerary.get("segments", []):
#                 row = {
#                     "departure_iata": segment["departure"]["iataCode"],
#                     "arrival_iata": segment["arrival"]["iataCode"],
#                     "departure_time": segment["departure"]["at"],
#                     "arrival_time": segment["arrival"]["at"],
#                     "carrier_code": segment["carrierCode"],
#                     "total_price": total_price,
#                     "cabin": cabin
#                 }
#                 rows.append(row)

#     return pd.DataFrame(rows)

# df = parse_flight_price(response.data)
# print(df)

## Database setting

In [4]:
# load all data we need
df_city = pd.read_csv('data/intermediate/city.csv')
df_weather = pd.read_csv('data/intermediate/weather.csv')
df_iata = pd.read_csv('data/intermediate/city_airport.csv')
routes_df = pd.read_csv('data/intermediate/routes.csv')
df_flight = pd.read_csv('data/intermediate/flight_month.csv')

In [15]:
df_flight["departure_date"] = pd.to_datetime(df_flight["departure_date"])

In [22]:
df_iata

,Unnamed: 0,id,iata_code,city_id
0,0,1,LAX,1
1,1,2,ORD,2
2,2,3,IAH,3
3,3,4,DEN,4
4,4,5,EWR,5
5,5,6,SFO,6
6,6,7,IAD,7


In [21]:
routes_df

,id,depar_airport_id,desti_airport_id
0,1,1,2
1,2,1,3
2,3,1,4
3,4,1,5
4,5,1,6
5,6,1,7
6,7,2,1
7,8,2,3
8,9,2,4
9,10,2,5


In [28]:
df_flight[df_flight["route_id"] < 7].groupby(['departure_date']).mean()

,id,route_id,price_quantile_minimum,price_quantile_low,price_quantile_middle,price_quantile_high,price_quantile_maximum
departure_date,,,,,,,
2025-01-31,16.0,3.5,55.645000,280.160000,334.040000,395.523333,515.445000
2025-02-28,17.0,3.5,55.645000,281.906667,339.303333,402.406667,514.390000
2025-03-31,18.0,3.5,55.645000,288.868333,351.393333,412.278333,520.261667
2025-04-30,19.0,3.5,56.198333,302.266667,367.860000,435.575000,527.133333
2025-05-31,20.0,3.5,56.315000,302.296667,376.543333,438.173333,533.805000
2025-06-30,21.0,3.5,57.636667,293.726667,355.403333,436.376667,528.095000


In [19]:
df_flight[df_flight["departure_date"] == pd.to_datetime("2025-05-31")].sort_values(by="price_quantile_middle", ascending=False)

,id,route_id,departure_date,price_quantile_minimum,price_quantile_low,price_quantile_middle,price_quantile_high,price_quantile_maximum
34,35,6,2025-05-31,87.57,359.83,557.96,602.62,715.37
58,59,11,2025-05-31,71.88,404.28,540.49,556.52,623.34
106,107,32,2025-05-31,66.92,431.74,518.77,579.86,661.51
94,95,29,2025-05-31,71.40,444.96,506.72,624.43,706.08
124,125,37,2025-05-31,104.65,424.84,491.99,655.11,657.23
22,23,4,2025-05-31,69.57,424.87,478.27,543.92,670.99
118,119,35,2025-05-31,66.85,467.97,476.25,580.36,694.33
82,83,25,2025-05-31,75.92,454.29,464.43,598.70,716.11
88,89,26,2025-05-31,75.54,392.25,447.39,591.67,676.99
4,5,1,2025-05-31,45.48,269.15,366.57,494.52,603.35


In [29]:
df_weather[df_weather['alert'] == True]

,id,date,city_id,weather_id,weather,alert
313,314,2025-05-16,2,502,Rain,True
347,348,2025-06-19,2,503,Rain,True
381,382,2025-01-27,3,502,Rain,True
404,405,2025-02-19,3,211,Thunderstorm,True
405,406,2025-02-19,3,502,Rain,True
409,410,2025-02-23,3,741,Fog,True
445,446,2025-03-29,3,211,Thunderstorm,True
758,244,2025-03-06,5,502,Rain,True
811,297,2025-04-27,5,502,Rain,True
905,391,2025-02-05,6,502,Rain,True


In [138]:
import duckdb as dd

# Create an in-memory DuckDB connection
con = dd.connect('data/static.duckdb')


In [ ]:
con.execute('SELECT f.price_quartile_middle FROM flight f JOIN route r ON f.route_id = r.id JOIN     nearest_airport depar_airport ON r.depar_airport_id = depar_airport.id JOIN     nearest_airport desti_airport ON r.desti_airport_id = desti_airport.id JOIN     city depar_city ON depar_airport.city_id = depar_city.id JOIN     city desti_city ON desti_airport.city_id = desti_city.id WHERE     depar_city.city_name = \'los angeles\'     AND desti_city.city_name = \'new york\' ORDER BY     f.departure_date DESC LIMIT 5;').fetchdf()

,price_quartile_middle


In [125]:
# optional: clear database
drop_order = ["flight", "weather", "route", "nearest_airport", "city"]
for table_name in drop_order:
    con.execute(f"DROP TABLE IF EXISTS {table_name}")

In [126]:
# Read SQL schema from file
with open("schema/static.sql", "r") as f:
    sql_script = f.read()

# Execute the schema to build tables
con.execute(sql_script)
# drop a useless table
con.execute("DROP TABLE IF EXISTS location")

# Verify tables were created
print(con.execute("SHOW TABLES").fetchdf())

              name
0             city
1           flight
2  nearest_airport
3            route
4          weather


In [127]:
## insert locations info
# Register and insert into the location table
con.register("df_city", df_city)
con.execute("INSERT INTO city (id, city_name) SELECT id, city_name FROM df_city")

con.execute("PRAGMA table_info('city')").fetchdf()

,cid,name,type,notnull,dflt_value,pk
0,0,id,BIGINT,True,None,True
1,1,city_name,VARCHAR,True,None,False


In [ ]:
## get all key info
# con.execute("SELECT * FROM duckdb_constraints()").fetchdf()

In [ ]:
# df_weather = df_weather.iloc[:, 1:]


In [ ]:
# df_weather_all['id'] = range(1, len(df_weather_all)+1)

In [107]:
df_weather_all

,id,date,city_id,weather_id,weather,alert
0,1,2025-01-01,1,804,Clouds,False
1,2,2025-01-02,1,800,Clear,False
2,3,2025-01-03,1,800,Clear,False
3,4,2025-01-04,1,800,Clear,False
4,5,2025-01-05,1,804,Clouds,False
...,...,...,...,...,...,...
8,1229,2025-06-13,3,800,Clear,False
9,1230,2025-06-14,3,800,Clear,False
10,1231,2025-06-15,3,803,Clouds,False
11,1232,2025-06-16,3,804,Clouds,False


In [128]:
### insert weather data
con.register("df_weather_all", df_weather_all)
con.execute("INSERT INTO weather SELECT * FROM df_weather_all")
con.execute("PRAGMA table_info('weather')").fetchdf()

,cid,name,type,notnull,dflt_value,pk
0,0,id,BIGINT,True,None,True
1,1,date,DATE,True,None,False
2,2,city_id,BIGINT,True,None,False
3,3,weather_id,BIGINT,True,None,False
4,4,weather,VARCHAR,True,None,False
5,5,alert,BOOLEAN,True,None,False


In [130]:
df_iata = df_iata.iloc[:, 1:]

In [131]:
### insert airport data
con.register("df_iata", df_iata)
con.execute("INSERT INTO nearest_airport SELECT * FROM df_iata")
con.execute("PRAGMA table_info('nearest_airport')").fetchdf()

,cid,name,type,notnull,dflt_value,pk
0,0,id,BIGINT,True,None,True
1,1,iata_code,VARCHAR,True,None,False
2,2,city_id,BIGINT,True,None,False


In [101]:
routes_df = routes_df.iloc[:, 1:]

In [132]:
### insert route data
con.register("routes_df", routes_df)
con.execute("INSERT INTO route SELECT * FROM routes_df")
con.execute("PRAGMA table_info('route')").fetchdf()

,cid,name,type,notnull,dflt_value,pk
0,0,id,BIGINT,True,None,True
1,1,depar_airport_id,BIGINT,True,None,False
2,2,desti_airport_id,BIGINT,True,None,False


In [ ]:
# df_flight

In [133]:
### insert holiday data
con.register("df_flight", df_flight)
con.execute("INSERT INTO flight SELECT * FROM df_flight")
con.execute("PRAGMA table_info('flight')").fetchdf()

,cid,name,type,notnull,dflt_value,pk
0,0,id,BIGINT,True,None,True
1,1,route_id,BIGINT,True,None,False
2,2,departure_date,DATE,True,None,False
3,3,price_quartile_minimum,SMALLINT,True,None,False
4,4,price_quartile_low,SMALLINT,True,None,False
5,5,price_quartile_middle,SMALLINT,True,None,False
6,6,price_quartile_high,SMALLINT,True,None,False
7,7,price_quartile_maximum,SMALLINT,True,None,False


In [134]:
tables = con.execute("SELECT name FROM sqlite_master WHERE type='table';").fetchall()

for (table_name,) in tables:
    print(f"Table: {table_name}")
    rows = con.execute(f"SELECT * FROM {table_name} LIMIT 3").fetchdf()
    print(rows)
    print("\n" + "-"*40 + "\n")

Table: city
   id    city_name
0   1  los angeles
1   2      chicago
2   3      houston

----------------------------------------

Table: flight
   id  route_id departure_date  price_quartile_minimum  price_quartile_low  \
0   1         1     2025-01-31                      45                 226   
1   2         1     2025-02-28                      45                 231   
2   3         1     2025-03-31                      45                 241   

   price_quartile_middle  price_quartile_high  price_quartile_maximum  
0                    323                  417                     591  
1                    327                  428                     589  
2                    343                  435                     595  

----------------------------------------

Table: nearest_airport
   id iata_code  city_id
0   1       LAX        1
1   2       ORD        2
2   3       IAH        3

----------------------------------------

Table: route
   id  depar_airport_id  desti_a

In [135]:

# Get all table names
tables = con.execute("SHOW TABLES").fetchall()
tables = [t[0] for t in tables]

# Print row count for each table
for table in tables:
    count = con.execute(f"SELECT COUNT(*) FROM {table}").fetchone()[0]
    print(f"Table '{table}' has {count} rows.")

Table 'city' has 7 rows.
Table 'df_city' has 7 rows.
Table 'df_flight' has 126 rows.
Table 'df_iata' has 7 rows.
Table 'df_weather_all' has 1233 rows.
Table 'flight' has 126 rows.
Table 'nearest_airport' has 7 rows.
Table 'route' has 42 rows.
Table 'routes_df' has 42 rows.
Table 'weather' has 1233 rows.


In [141]:
# end the database
# con.commit()  # Optional, but ensures changes are flushed
con.close()   # Closes the DB and safely handles the WAL